<a href="https://colab.research.google.com/github/lisabroadhead/data_science_machine-learning/blob/main/pre_processing_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Processing Exercise (Practice)
- Lisa Broadhead
- June 22, 2022

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [2]:
file = '/content/drive/MyDrive/Colab Notebooks/coding_dojo/Machine Learning/files/insurance.csv'

In [4]:
df = pd.read_csv(file)
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Define features (X) and target (y)

In [5]:
y = df['charges']
X = df.drop(columns='charges')

In [6]:
y.head()

0    16884.92400
1     1725.55230
2     4449.46200
3    21984.47061
4     3866.85520
Name: charges, dtype: float64

In [7]:
X.head()

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest


### Train test split the data to prepare for machine learning

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [10]:
len(df), len(X_train), len(X_test)

(1338, 1003, 335)

### Identify each feature as numerical, ordinal, or nominal. (Please provide this answer in a text cell in your Colab notebook)

1. age - Numeric
2. sex - Nominal 
3. bmi - Numeric 
4. childern - Numeric 
5. smoker - Nominal 
6. region - Nominal 

### Ordinal encode any ordinal features

- Skipped: Didn't think any of thw categories were ordinal

### One Hot Encode any nominal features 

In [13]:
cat_selector = make_column_selector(dtype_include='object')

In [14]:
cat_selector(X_train)

['sex', 'smoker', 'region']

In [24]:
train_cat_data = X_train[cat_selector(X_train)]
test_cat_data = X_test[cat_selector(X_test)]
train_cat_data

,sex,smoker,region
693,male,no,northwest
1297,female,no,southeast
634,male,no,southwest
1022,male,yes,southeast
178,female,no,southwest
...,...,...,...
1095,female,no,northeast
1130,female,no,southeast
1294,male,no,northeast
860,female,yes,southwest


In [22]:
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_encoder.fit(train_cat_data)

train_ohe = ohe_encoder.transform(train_cat_data)
test_ohe = ohe_encoder.transform(test_cat_data) 

train_ohe 

array([[0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.]])

### Scale any numeric features

In [29]:
ohe_column_names = ohe_encoder.get_feature_names_out(train_cat_data.columns)

train_ohe = pd.DataFrame(train_ohe, columns=ohe_column_names)
test_ohe = pd.DataFrame(test_ohe, columns=ohe_column_names)

train_ohe

,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
998,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
999,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1000,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1001,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


### Concatenate all features back into one dataframe.

In [35]:
num_selector = make_column_selector(dtype_include='number')

train_nums = X_train[num_selector(X_train)].reset_index(drop=True)
test_nums = X_test[num_selector(X_test)].reset_index(drop=True)

X_train_processed = pd.concat([train_nums, train_ohe], axis=1)
X_test_processed = pd.concat([test_nums,test_ohe], axis=1)

X_train_processed

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,24,23.655,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,28,26.510,2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,51,39.700,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,47,36.080,1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,46,28.900,2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
998,18,31.350,4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
999,39,23.870,5,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1000,58,25.175,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1001,37,47.600,2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
